# EDA: Modelo Hedonic MICRO - Gràcia

**Issue**: #202 - Fase 2  
**Fecha**: 2025-12-19  
**Objetivo**: Análisis exploratorio de datos matched (Catastro + Idealista) para modelo hedonic MICRO

---

## 📊 Datasets

- **Catastro**: 731 inmuebles reales de Gràcia
- **Idealista**: 100 propiedades mock (simuladas)
- **Matched**: 100 observaciones combinadas

---

## 🎯 Objetivos del EDA

1. Visualizar distribuciones de variables clave
2. Analizar correlaciones entre features y precio
3. Identificar outliers y valores atípicos
4. Explorar relaciones precio vs características
5. Comparar distribuciones por barrio
6. Generar insights para mejorar el modelo


In [1]:
# Imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
from pathlib import Path

# Configuración
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Rutas
PROJECT_ROOT = Path().resolve().parents[1]
DATA_DIR = PROJECT_ROOT / "spike-data-validation" / "data" / "processed" / "fase2"

print(f"📁 Directorio de datos: {DATA_DIR}")


📁 Directorio de datos: /Users/adrianiraeguialvear/Projects/barcelona-housing-demographics-analyzer/spike-data-validation/data/processed/fase2


In [2]:
# Cargar datos matched
df_matched = pd.read_csv(DATA_DIR / "catastro_idealista_matched.csv")

print(f"✅ Datos cargados: {len(df_matched)} observaciones")
print(f"\nColumnas disponibles ({len(df_matched.columns)}):")
print(df_matched.columns.tolist())

# Información básica
print(f"\n📊 Información del dataset:")
print(df_matched.info())


✅ Datos cargados: 100 observaciones

Columnas disponibles (42):
['ref_base', 'referencia_catastral_catastro', 'direccion_normalizada', 'superficie_m2', 'ano_construccion', 'plantas', 'lat', 'lon', 'direccion', 'barrio_id_catastro', 'barrio_nombre', 'propertyId', 'url', 'price', 'priceByArea', 'size', 'rooms', 'bathrooms', 'floor', 'exterior', 'elevator', 'parkingSpace', 'condition', 'orientation', 'address', 'district', 'neighborhood', 'latitude', 'longitude', 'yearBuilt', 'description', 'publicationDate', 'operation', 'propertyType', 'referencia_catastral_idealista', 'barrio_id_idealista', 'precio', 'barrio_id', 'habitaciones', 'banos', 'ascensor', 'precio_m2']

📊 Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 42 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ref_base                        100 non-null    object 
 1   referenci

## 1. Estadísticas Descriptivas


In [5]:
# Variables clave para análisis
key_vars = ['precio_m2', 'precio', 'superficie_m2', 'ano_construccion', 'plantas', 
            'habitaciones', 'banos', 'barrio_id']

# Filtrar variables disponibles
available_vars = [v for v in key_vars if v in df_matched.columns]

print("📊 Estadísticas descriptivas:")
print(df_matched[available_vars].describe())

import plotly.express as px

def plot_key_var_summaries(df: pd.DataFrame, variables: list) -> None:
    """
    Muestra un gráfico de resumen de estadísticas descriptivas para variables clave.

    Args:
        df: DataFrame de datos de matched catastro/idealista.
        variables: Lista de variables numéricas a visualizar.
    """
    # Filtrar solo variables numéricas válidas
    num_vars = [col for col in variables if pd.api.types.is_numeric_dtype(df[col])]

    # Calcular estadísticas
    desc = df[num_vars].describe().T[['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']]
    desc = desc.reset_index().rename(columns={'index': 'Variable'})

    # Redondear para visualización
    desc = desc.round({'mean': 1, 'std': 1, 'min': 1, '25%': 1, '50%': 1, '75%': 1, 'max': 1})

    # Gráfico de resumen (heatmap)
    fig = px.imshow(
        desc.set_index('Variable').T,
        text_auto=True,
        color_continuous_scale="Blues",
        aspect="auto",
        title="Resumen Estadístico de Variables Clave"
    )
    fig.update_layout(
        xaxis_title="Variable",
        yaxis_title="Estadística"
    )
    fig.show()

# Visualizar resumen de variables clave
plot_key_var_summaries(df_matched, available_vars)


📊 Estadísticas descriptivas:
         precio_m2        precio  superficie_m2  ano_construccion     plantas  \
count   100.000000  1.000000e+02      100.00000        100.000000  100.000000   
mean   4678.562650  3.597677e+05       77.81306       1975.030000    1.400000   
std     657.454465  3.276845e+05       72.61428         25.852126    2.441228   
min    3302.416667  1.108000e+04        2.92200       1900.000000   -3.000000   
25%    4179.763644  1.177092e+05       26.50000       1966.000000    0.000000   
50%    4619.989524  3.250750e+05       76.00000       1973.000000    1.000000   
75%    5251.619048  4.671932e+05       97.75000       1983.000000    3.000000   
max    6360.545455  2.107993e+06      473.00000       2017.000000    7.000000   

       habitaciones       banos   barrio_id  
count    100.000000  100.000000  100.000000  
mean       2.440000    1.570000   30.820000  
std        0.856585    0.685418    1.242188  
min        1.000000    1.000000   28.000000  
25%        

## 2. Distribuciones de Variables Clave


In [6]:
# Distribución de precio/m²
fig = px.histogram(
    df_matched,
    x='precio_m2',
    nbins=30,
    title='Distribución de Precio/m² (€/m²)',
    labels={'precio_m2': 'Precio por m² (€/m²)', 'count': 'Frecuencia'}
)
fig.add_vline(
    x=df_matched['precio_m2'].mean(),
    line_dash="dash",
    line_color="red",
    annotation_text=f"Media: {df_matched['precio_m2'].mean():.0f} €/m²"
)
fig.show()


In [7]:
# Distribución de superficie
fig = px.histogram(
    df_matched,
    x='superficie_m2',
    nbins=30,
    title='Distribución de Superficie (m²)',
    labels={'superficie_m2': 'Superficie (m²)', 'count': 'Frecuencia'}
)
fig.add_vline(
    x=df_matched['superficie_m2'].mean(),
    line_dash="dash",
    line_color="red",
    annotation_text=f"Media: {df_matched['superficie_m2'].mean():.1f} m²"
)
fig.show()

# Identificar outliers
Q1 = df_matched['superficie_m2'].quantile(0.25)
Q3 = df_matched['superficie_m2'].quantile(0.75)
IQR = Q3 - Q1
outliers = df_matched[(df_matched['superficie_m2'] < Q1 - 1.5*IQR) | 
                      (df_matched['superficie_m2'] > Q3 + 1.5*IQR)]
print(f"\n⚠️  Outliers en superficie: {len(outliers)} observaciones")
if len(outliers) > 0:
    print(outliers[['superficie_m2', 'precio_m2', 'barrio_id']].head())



⚠️  Outliers en superficie: 4 observaciones
    superficie_m2    precio_m2  barrio_id
3           324.0  4271.123457         32
28          289.0  4688.404844         32
38          473.0  4456.644820         32
41          380.0  4342.371053         28


In [9]:
# Distribución por barrio
if 'barrio_nombre' in df_matched.columns:
    barrio_col = 'barrio_nombre'
elif 'barrio_id' in df_matched.columns:
    # Crear nombres de barrio si no existen
    barrio_map = {
        28: 'Vallcarca i els Penitents',
        29: 'el Coll',
        30: 'la Salut',
        31: 'la Vila de Gràcia',
        32: 'el Camp d\'en Grassot i Gràcia Nova'
    }
    df_matched['barrio_nombre'] = df_matched['barrio_id'].map(barrio_map)
    barrio_col = 'barrio_nombre'

fig = px.box(
    df_matched,
    x=barrio_col,
    y='precio_m2',
    title='Distribución de Precio/m² por Barrio',
    labels={'precio_m2': 'Precio por m² (€/m²)', barrio_col: 'Barrio'}
)
fig.update_xaxes(tickangle=45)
fig.show()

# Análisis detallado de outliers en superficie
if len(outliers) > 0:
    print("\n" + "="*70)
    print("🔍 ANÁLISIS DETALLADO DE OUTLIERS EN SUPERFICIE")
    print("="*70)
    
    outlier_cols = ['superficie_m2', 'precio_m2', 'precio', 'habitaciones', 'banos', 
                    'barrio_id', 'ano_construccion', 'plantas']
    outlier_cols = [c for c in outlier_cols if c in outliers.columns]
    
    print("\n📊 Características de los outliers:")
    print(outliers[outlier_cols].to_string())
    
    print("\n📈 Comparación con el resto del dataset:")
    df_normal = df_matched[~df_matched.index.isin(outliers.index)]
    print(f"  Superficie media (sin outliers): {df_normal['superficie_m2'].mean():.1f} m²")
    print(f"  Superficie media (outliers):     {outliers['superficie_m2'].mean():.1f} m²")
    print(f"  Ratio: {outliers['superficie_m2'].mean() / df_normal['superficie_m2'].mean():.1f}x")
    
    print(f"\n  Precio/m² medio (sin outliers): {df_normal['precio_m2'].mean():.0f} €/m²")
    print(f"  Precio/m² medio (outliers):     {outliers['precio_m2'].mean():.0f} €/m²")
    
    print("\n💡 Interpretación:")
    print("  - Pisos >300 m² son muy raros en Gràcia (típico: 60-90 m²)")
    print("  - Podrían ser:")
    print("    • Áticos o dúplex grandes")
    print("    • Errores en datos Catastro (superficie total del edificio)")
    print("    • Casos especiales (lofts, naves reconvertidas)")
    print("  - Recomendación: Revisar manualmente o filtrar si son errores")



🔍 ANÁLISIS DETALLADO DE OUTLIERS EN SUPERFICIE

📊 Características de los outliers:
    superficie_m2    precio_m2   precio  habitaciones  banos  barrio_id  ano_construccion  plantas
3           324.0  4271.123457  1383844             2      1         32            2017.0        1
28          289.0  4688.404844  1354949             2      1         32            1973.0        0
38          473.0  4456.644820  2107993             4      3         32            1966.0       -1
41          380.0  4342.371053  1650101             2      1         28            1970.0        0

📈 Comparación con el resto del dataset:
  Superficie media (sin outliers): 65.8 m²
  Superficie media (outliers):     366.5 m²
  Ratio: 5.6x

  Precio/m² medio (sin outliers): 4689 €/m²
  Precio/m² medio (outliers):     4440 €/m²

💡 Interpretación:
  - Pisos >300 m² son muy raros en Gràcia (típico: 60-90 m²)
  - Podrían ser:
    • Áticos o dúplex grandes
    • Errores en datos Catastro (superficie total del edificio)

## 3. Correlaciones


In [13]:
# Matriz de correlaciones
numeric_cols = [
    'precio_m2', 'precio', 'superficie_m2', 'ano_construccion',
    'plantas', 'habitaciones', 'banos'
]
numeric_cols = [c for c in numeric_cols if c in df_matched.columns]

corr_matrix = df_matched[numeric_cols].corr()

# Mapeo de nombres claros y unidades para los ejes
col_labels = {
    'precio_m2': 'Precio/m² (€/m²)',
    'precio': 'Precio total (€)',
    'superficie_m2': 'Superficie (m²)',
    'ano_construccion': 'Año construcción',
    'plantas': 'Plantas (nº)',
    'habitaciones': 'Habitaciones (nº)',
    'banos': 'Baños (nº)'
}
axis_labels = [col_labels.get(col, col) for col in corr_matrix.columns]
axis_labels_rows = [col_labels.get(row, row) for row in corr_matrix.index]

fig = px.imshow(
    corr_matrix,
    text_auto=".2f",
    aspect="auto",
    title="🔗 Matriz de Correlaciones entre Variables Inmobiliarias",
    color_continuous_scale="RdBu",
    zmin=-1,
    zmax=1,
    labels=dict(
        color="Coef. correlación"
    ),
    x=axis_labels,
    y=axis_labels_rows
)

fig.update_layout(
    title={
        'text': "🔗 Matriz de Correlaciones entre Características de Vivienda (con unidades)",
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title="Variables (con unidades)",
    yaxis_title="Variables (con unidades)",
    font=dict(size=13),
    width=940,
    height=650,
    coloraxis_colorbar=dict(
        title="Coef. de<br>correlación",
        tickvals=[-1, -0.5, 0, 0.5, 1],
        ticktext=["-1<br>(negativa fuerte)", "-0.5", "0", "0.5", "1<br>(positiva fuerte)"]
    ),
)
fig.update_xaxes(side="bottom", tickangle=45)

fig.show()

print("\n📊 Correlaciones con precio_m2 (Coef. de correlación de Pearson):")
corr_with_price = corr_matrix['precio_m2'].sort_values(ascending=False)
for var, corr in corr_with_price.items():
    if var != 'precio_m2':
        nombre = col_labels.get(var, var)
        print(f"  {nombre:30s}: {corr:7.3f}")



📊 Correlaciones con precio_m2 (Coef. de correlación de Pearson):
  Año construcción              :   0.212
  Precio total (€)              :   0.038
  Plantas (nº)                  :  -0.053
  Superficie (m²)               :  -0.091
  Baños (nº)                    :  -0.181
  Habitaciones (nº)             :  -0.223


## 4. Relaciones Precio vs Características


In [16]:
# ⚡ Relación entre Precio/m² y Superficie de Vivienda
# Este gráfico de dispersión muestra cómo varía el precio por metro cuadrado (€/m²) 
# en función de la superficie del inmueble (m²). Cada punto representa una vivienda.
# Los colores distinguen barrios y el tamaño del punto (si disponible) refleja el precio total.
# Consejo: Use el hover para ver habitaciones y año de construcción de cada vivienda.

fig = px.scatter(
    df_matched,
    x='superficie_m2',
    y='precio_m2',
    color='barrio_id' if 'barrio_id' in df_matched.columns else None,
    size='precio' if 'precio' in df_matched.columns else None,
    hover_data=['habitaciones', 'ano_construccion'] if all(
        c in df_matched.columns for c in ['habitaciones', 'ano_construccion']
    ) else None,
    title='Precio/m² vs Superficie (Cada punto: una vivienda)',
    labels={
        'superficie_m2': 'Superficie (m²)',
        'precio_m2': 'Precio por metro cuadrado (€ / m²)',
        'barrio_id': 'Barrio',
        'precio': 'Precio total (€)',
        'habitaciones': 'Habitaciones',
        'ano_construccion': 'Año construcción'
    }
)
fig.update_traces(marker=dict(sizemin=4, opacity=0.75), selector=dict(mode='markers'))
fig.update_layout(
    xaxis=dict(title='Superficie (m²)'),
    yaxis=dict(title='Precio/m² (€/m²)'),
    legend_title_text='Barrio',
    hoverlabel=dict(bgcolor='white', font_size=11, font_family='Arial'),
    font=dict(size=13),
    width=850,
    height=500,
    margin=dict(l=60, r=30, t=60, b=60)
)
fig.show()

# Calcular y mostrar la correlación entre precio/m2 y superficie, explicando el valor:
if 'superficie_m2' in df_matched.columns:
    corr_sup = df_matched[['precio_m2', 'superficie_m2']].corr().iloc[0, 1]
    print("\n📈 Correlación (coeficiente de Pearson) entre precio/m² y superficie:")
    print(f"   Un valor negativo indica que, en promedio, viviendas más grandes tienden a tener un precio/m² más bajo.")
    print(f"   Correlación precio_m2 vs superficie_m2: {corr_sup:.3f}")



📈 Correlación (coeficiente de Pearson) entre precio/m² y superficie:
   Un valor negativo indica que, en promedio, viviendas más grandes tienden a tener un precio/m² más bajo.
   Correlación precio_m2 vs superficie_m2: -0.091


In [18]:
# Precio vs Año construcción
import plotly.express as px
import pandas as pd

# 1. Preparación de datos y filtrado de nulos
cols_interes = ['ano_construccion', 'precio_m2']
df_plot = df_matched.dropna(subset=cols_interes).copy()

if not df_plot.empty:
    # 2. Creación del gráfico con mejores prácticas
    fig = px.scatter(
        df_plot,
        x='ano_construccion',
        y='precio_m2',
        color='barrio_id' if 'barrio_id' in df_plot.columns else None,
        size='superficie_m2' if 'superficie_m2' in df_plot.columns else None,
        opacity=0.6,
        trendline="ols",  # Añade línea de tendencia lineal
        marginal_x="histogram",  # Distribución de años de construcción
        marginal_y="violin",     # Distribución de precios
        template='plotly_white', # Fondo limpio para mejor legibilidad
        title='<b>Relación entre Antigüedad y Valor de Mercado</b><br><span style="font-size: 12px;">Análisis de precio por metro cuadrado vs. año de construcción (Etiqueta <ACO>)</span>',
        labels={
            'ano_construccion': 'Año de Construcción',
            'precio_m2': 'Precio (€/m²)',
            'barrio_id': 'Identificador de Barrio',
            'superficie_m2': 'Superficie (m²)'
        },
        hover_data=['superficie_m2'] # Datos extra al pasar el ratón
    )

    # 3. Ajustes finos de formato
    fig.update_layout(
        title_font_size=20,
        legend_title_text='Barrios',
        xaxis=dict(gridcolor='lightgrey'),
        yaxis=dict(gridcolor='lightgrey', tickprefix="€")
    )
    
    fig.show()
    
    # 4. Cálculo de correlación de Pearson (r)
    corr_ano = df_plot['precio_m2'].corr(df_plot['ano_construccion'])
    
    print("-" * 50)
    print(f"ANÁLISIS ESTADÍSTICO:")
    print(f"Correlación de Pearson (r): {corr_ano:.3f}")
    if abs(corr_ano) < 0.3:
        print("Interpretación: Existe una correlación débil entre la edad y el precio.")
    else:
        print("Interpretación: Existe una correlación significativa entre la edad y el precio.")
    print("-" * 50)

--------------------------------------------------
ANÁLISIS ESTADÍSTICO:
Correlación de Pearson (r): 0.212
Interpretación: Existe una correlación débil entre la edad y el precio.
--------------------------------------------------


In [35]:
if 'habitaciones' in df_matched.columns:
    df_plot_hab = df_matched.dropna(subset=['habitaciones', 'precio_m2']).copy()
    habitaciones_sorted = sorted(df_plot_hab['habitaciones'].unique())

    # Calcular la mediana del precio_m2 por número de habitaciones
    mediana_por_hab = (
        df_plot_hab.groupby('habitaciones')['precio_m2'].median().round(0)
    )

    # Definir paleta igual que la del gráfico (Safe)
    colors_palette = px.colors.qualitative.Safe
    color_map = {hab: colors_palette[i % len(colors_palette)] for i, hab in enumerate(habitaciones_sorted)}

    fig = px.box(
        df_plot_hab,
        x='habitaciones',
        y='precio_m2',
        color='habitaciones',
        points='all',
        category_orders={'habitaciones': habitaciones_sorted},
        color_discrete_sequence=colors_palette,
        title='<b>Distribución de Precio/m² por Número de Habitaciones</b>',
        labels={'habitaciones': 'Habitaciones', 'precio_m2': 'Precio/m² (€)'}
    )

    # Ajustes visuales generales
    fig.update_layout(
        template="plotly_white",
        height=650,
        margin=dict(t=140, b=80, l=80, r=320),  # margen derecho extra para leyenda personalizada
        title=dict(y=0.92, x=0.05, xanchor='left'),
        showlegend=False,
        xaxis=dict(
            title_standoff=30,
            tickfont=dict(size=12),
            showgrid=True,
            gridcolor="lightgrey",
            gridwidth=1,
            zeroline=False
        ),
        yaxis=dict(
            title_standoff=30,
            tickprefix='€',
            tickfont=dict(size=12),
            showgrid=True,
            gridcolor="lightgrey",
            gridwidth=1,
            zeroline=False
        )
    )

    # Anotación general superior
    fig.add_annotation(
        text=(
            "<b>Análisis de Segmentos:</b> Cada caja representa un grupo de inmuebles.<br>"
            "Observe cómo las muescas indican la confianza en la mediana del valor."
        ),
        xref="paper", yref="paper",
        x=1.2, y=1.2,
        showarrow=False,
        align="left",
        bgcolor="rgba(255, 255, 255, 0.9)",
        bordercolor="lightgray",
        borderwidth=1,
    )

    # Preparar texto/leyenda de medianas para cada cluster (habitaciones)
    leyenda_texto = "<b>Precio/m² mediano por grupo:</b><br>"
    for i, hab in enumerate(habitaciones_sorted):
        valor_mediana = mediana_por_hab[hab]
        color_celda = color_map[hab]
        leyenda_texto += (
            f"<span style='display:inline-block;width:13px;height:13px;"
            f"background:{color_celda};margin-right:6px;border-radius:2px;'>&nbsp;</span>"
            f"<span style='font-weight:bold;'>{hab} hab.</span>: "
            f"<span style='color:{color_celda};'>€{valor_mediana:,.0f}</span><br>"
        )

    # Añadir la "leyenda" como anotación fija al margen derecho fuera de la zona del gráfico
    fig.add_annotation(
        text=leyenda_texto,
        xref="paper", yref="paper",
        x=1.25, y=0.5,  # fuera del eje derecho, centrado vertical
        showarrow=False,
        align="left",
        bordercolor="lightgray",
        borderwidth=1,
        bgcolor="rgba(255,255,255,0.92)",
        font=dict(size=13),
    )

    fig.update_traces(jitter=0.4, marker_size=4)
    fig.show()

## 5. Análisis por Barrio


In [38]:
# Estadísticas por barrio (con visualización horizontal mejorada y anotaciones)
if 'barrio_id' in df_matched.columns and barrio_col in df_matched.columns:
    import plotly.graph_objects as go
    import numpy as np

    # Calcular estadísticas por barrio
    stats_barrio = (
        df_matched.groupby(['barrio_id', barrio_col])  # Mantener id y nombre
        .agg(
            precio_m2_mean=('precio_m2', 'mean'),
            precio_m2_std=('precio_m2', 'std'),
            n=('precio_m2', 'count'),
            superficie_m2_mean=('superficie_m2', 'mean') if 'superficie_m2' in df_matched.columns else ('precio_m2', 'count'),  # fallback
            precio_mean=('precio', 'mean') if 'precio' in df_matched.columns else ('precio_m2', 'count')
        )
        .reset_index()
        .round(2)
    )

    print("📊 Estadísticas por barrio:")
    print(stats_barrio)

    # Ordenar barrios por precio_m2 descendente
    stats_barrio = stats_barrio.sort_values('precio_m2_mean', ascending=False)

    # Preparar nombres con (n=XX)
    stats_barrio['barrio_label'] = stats_barrio[barrio_col] + " (n=" + stats_barrio['n'].astype(str) + ")"

    # Precio medio para línea de referencia (sobre todas las muestras)
    precio_medio_bcna = np.round(df_matched['precio_m2'].mean(), 2)

    # Construir gráfico horizontal
    fig = go.Figure()

    fig.add_trace(go.Bar(
        y=stats_barrio['barrio_label'],
        x=stats_barrio['precio_m2_mean'],
        orientation='h',
        marker=dict(color='rgba(44, 160, 101, 0.85)'),
        hovertemplate="<b>%{y}</b><br>Precio/m²: %{x:.0f} €<br>Muestra: %{y}<extra></extra>"
    ))

    # Añadir etiquetas con precio exacto al final de cada barra
    for idx, row in stats_barrio.iterrows():
        fig.add_annotation(
            x=row['precio_m2_mean'],
            y=row['barrio_label'],
            text=f"{row['precio_m2_mean']:,.0f} €/m²",
            showarrow=False,
            xanchor="left",
            yanchor="middle",
            align="left",
            font=dict(size=13, color="rgb(44, 44, 44)"),
            bgcolor="rgba(255,255,255,0.0)",
            xshift=8
        )

    # Línea de referencia de precio medio general
    fig.add_vline(
        x=precio_medio_bcna,
        line_dash="dash",
        line_color="crimson",
        annotation_text=f"Media BCN: {precio_medio_bcna:,.0f} €/m²",
        annotation_position="top right",
        annotation_font_color="crimson"
    )

    # Limpiar estética: ejes y cuadrícula
    fig.update_layout(
        title="Precio/m² Medio por Barrio<br><sup>Con tamaño de muestra (catastro: LBI) y referencia municipal</sup>",
        xaxis=dict(
            title="Precio/m² Medio (€/m²)",
            gridcolor="rgba(0,0,0,0)",
            zeroline=True, rangemode="tozero"
        ),
        yaxis=dict(
            title="Barrio",
            gridcolor="rgba(0,0,0,0)",
            automargin=True
        ),
        plot_bgcolor="white",
        height=100 + 28 * len(stats_barrio),  # Ajuste dinámico
        margin=dict(l=220, r=70, t=70, b=30),
        showlegend=False
    )

    # Asegurar escala real: eje X inicia en 0, sin gridlines molestos
    max_precio = int((stats_barrio['precio_m2_mean'].max() // 500 + 2) * 500)
    fig.update_xaxes(range=[0, max_precio], showgrid=False, zeroline=True, dtick=500)
    fig.update_yaxes(showgrid=False)

    fig.show()


📊 Estadísticas por barrio:
   barrio_id                       barrio_nombre  precio_m2_mean  \
0         28           Vallcarca i els Penitents         4391.39   
1         29                             el Coll         4041.53   
2         30                            la Salut         4610.62   
3         31                   la Vila de Gràcia         5122.56   
4         32  el Camp d'en Grassot i Gràcia Nova         4756.39   

   precio_m2_std   n  superficie_m2_mean  precio_mean  
0         707.28   5               90.88    392085.40  
1         620.66  10               84.40    348475.40  
2         585.74  27               49.22    219833.26  
3         747.92  14               57.14    289592.64  
4         560.53  44               98.95    466858.89  


## 6. Visualización Espacial (Mapa)


In [43]:
import plotly.express as px
import plotly.graph_objects as go

# 1. Filtrar y limpiar datos solo de Gràcia y superficies > 20 m²
cols_mapa = ['latitude', 'longitude', 'precio_m2', 'barrio_nombre']
if all(col in df_matched.columns for col in cols_mapa):
    # Mantener solo registros del barrio de Gràcia, superficie mínima y sin nulos clave
    df_map = df_matched.dropna(subset=cols_mapa).copy()
    # Normaliza y filtra a Gràcia
    mask_gracia = df_map['barrio_nombre'].str.lower().str.contains("gràcia")
    df_map = df_map[mask_gracia]
    if 'superficie_m2' in df_map.columns:
        df_map = df_map[df_map['superficie_m2'] > 20]
    else:
        df_map['superficie_m2'] = None  # asegura columna aunque sea None

    if len(df_map) > 0:
        # Definir centro geográfico medio para Gràcia (más robusto si hay dispersión)
        lat_center = df_map['latitude'].mean()
        lon_center = df_map['longitude'].mean()
        
        # --- AMPLIAR RADIO DE LOS PUNTOS EN EL MAPA ---
        # Por defecto, size_max=18 es bajo para aparentar menor muestra. Lo elevamos a mejorar visibilidad.
        size_max_mapa = 52  # Puede ajustarse (20-40) según densidad/zoom

        fig = px.scatter_mapbox(
            df_map,
            lat='latitude',
            lon='longitude',
            color='precio_m2',
            size='superficie_m2',
            color_continuous_scale='Viridis',
            size_max=size_max_mapa,  # Radio de punto ampliado
            zoom=14.2,
            center=dict(lat=lat_center, lon=lon_center),  # Centro dinámico en Gràcia
            mapbox_style='carto-positron',
            custom_data=['barrio_nombre', 'direccion', 'habitaciones', 'superficie_m2', 'precio', 'precio_m2'],
            labels={
                'precio_m2': 'Precio (€/m²)', 
                'superficie_m2': 'Superficie (m²)'
            },
            title=None  # Sin título aquí, se añade con fig.update_layout
        )

        # Personalización del hover: muestra dirección si existe y datos relevantes
        fig.update_traces(
            hovertemplate=(
                "<b>Barrio:</b> %{customdata[0]}<br>"
                "<b>Dirección:</b> %{customdata[1]}<br>"
                "<b>Precio total:</b> %{customdata[4]:,.0f} €<br>"
                "<b>Precio/m²:</b> %{customdata[5]:,.0f} €/m²<br>"
                "<b>Hab:</b> %{customdata[2]}<br>"
                "<b>Superficie:</b> %{customdata[3]} m²"
                "<extra></extra>"
            )
        )

        fig.update_layout(
            height=max(600, min(1200, 34 * len(df_map))),  # Escala dinámica para no saturar
            margin=dict(t=120, b=30, l=16, r=16),
            mapbox=dict(
                bearing=10,  # Pequeño tilt para dar sensación 3D
                pitch=15
            ),
            title=dict(
                text=(
                    "<b>Mapa de Precios (<i>€/m²</i>) en Barcelona: Barrio de Gràcia (>20 m²)</b><br>"
                    "<span style='font-size:14px'>"
                    "Cada punto corresponde a una vivienda geolocalizada. "
                    "Color = Precio por metro cuadrado. Tamaño = Superficie.<br>"
                    "Interactúe para detalle y zoom.</span>"
                ),
                font=dict(size=22),
                y=0.98, x=0.01, xanchor='left'
            ),
            coloraxis_colorbar=dict(
                title="€/m²",
                title_side="top",
                thickness=17,
                lenmode="fraction", len=0.7,
                yanchor="middle", y=0.5,
                outlinewidth=1,
                xpad=18
            )
        )

        # Añadir anotación de contexto (puede colocarse sobre el mapa)
        fig.add_annotation(
            text=(
                "<b>💡 Nota:</b> Solo visualiza inmuebles con superficie útil &gt; 20m² en Gràcia.<br>"
                "<i>Colores oscuros → valores superiores.</i>"
            ),
            xref="paper", yref="paper", x=0.01, y=0.005,
            showarrow=False, align="left",
            font=dict(size=13),
            bgcolor="rgba(255,255,255,0.95)",
            bordercolor="gray", borderwidth=1,
        )

        # Añadir escala de zoom mínima y máxima ajustada
        fig.update_mapboxes(
            bearing=0,
            pitch=10,
            zoom=14.2,
            center=dict(lat=lat_center, lon=lon_center)
        )

        # Mejorar contraste para usuarios con daltonismo
        fig.update_layout(
            coloraxis_colorscale="Viridis",
        )

        fig.show()
    else:
        import logging
        logging.getLogger(__name__).warning(
            "No hay datos válidos para generar el mapa de Gràcia con superficie mayor a 20 m²."
        )

/var/folders/8t/1rzjlj6n3rj3v08skl2pwjbm0000gn/T/ipykernel_69290/1422893748.py:26: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



## 7. Análisis de Outliers


In [64]:
from typing import Tuple
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import logging

logger = logging.getLogger(__name__)

def detect_outliers_iqr(df: pd.DataFrame, column: str) -> Tuple[pd.DataFrame, float, float]:
    """
    Detecta outliers usando el método IQR.
    
    Args:
        df: DataFrame con los datos.
        column: Nombre de la columna numérica a analizar.
    
    Returns:
        outliers: DataFrame con registros outlier según IQR.
        lower_bound: Límite inferior.
        upper_bound: Límite superior.
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Outliers en precio_m2
outliers_precio, lb_precio, ub_precio = detect_outliers_iqr(df_matched, 'precio_m2')
print(f"⚠️  Outliers en precio_m2: {len(outliers_precio)} observaciones")
print(f"   Rango normal esperado: [{lb_precio:.2f}, {ub_precio:.2f}] €/m²")

# Outliers en superficie
if 'superficie_m2' in df_matched.columns:
    outliers_sup, lb_sup, ub_sup = detect_outliers_iqr(df_matched, 'superficie_m2')
    print(f"\n⚠️  Outliers en superficie_m2: {len(outliers_sup)} observaciones")
    print(f"   Rango normal esperado: [{lb_sup:.2f}, {ub_sup:.2f}] m²")
    
    if len(outliers_sup) > 0:
        # Crear columna 'es_outlier' de forma segura
        df_matched = df_matched.copy()
        df_matched['es_outlier'] = df_matched.index.isin(outliers_sup.index)

        # --- REESCRITURA: Mejorar visualización para evitar solapamiento y ampliar eje y ---
        # Ajustar tamaño de marcador y opacidad para evitar superposición excesiva
        size_col = 'precio' if 'precio' in df_matched.columns else None
        marker_kwargs = {
            'size_max': 18,            # limitar el tamaño máximo
            'opacity': 0.6,            # añadir opacidad
        }
        # reducción del tamaño base marca
        custom_marker = dict(size=8, opacity=0.6, line=dict(width=0.7, color='rgba(100,100,100,0.2)'))

        fig = px.scatter(
            df_matched,
            x='superficie_m2',
            y='precio_m2',
            color='es_outlier',
            size=size_col,
            hover_data=['barrio_id', 'habitaciones', 'ano_construccion'] if all(
                c in df_matched.columns for c in ['barrio_id', 'habitaciones', 'ano_construccion']
            ) else None,
            title=(
                "🔴 <b>¿Excepciones o errores? Outliers en Superficie frente a Precio/m²</b><br>"
                "<span style='font-size:13px'>Observa que los puntos rojos quedan fuera del rango "
                "normal para la superficie construida.<br>"
              
            ),
            labels={
                'superficie_m2': 'Superficie (m²)', 
                'precio_m2': 'Precio/m² (€/m²)', 
                'es_outlier': 'Es outlier'
            },
            color_discrete_map={True: '#d62728', False: '#1f77b4'},
            **marker_kwargs if size_col is None else {}
        )

        # Reducir el tamaño de los puntos por defecto para ambos grupos
        fig.update_traces(
            marker=custom_marker,
            selector=dict(mode='markers'),
        )

        # Mejorar la superposición: cuando hay muchos valores similares, se puede usar jitter.
        # Jitter solo sobre x o y si la distribución es muy apretada
        # --- Jitter manual al eje x e y ---
        import numpy as np
        jitter_strength = 0.4  # metros cuadrados (x)
        jitter_y_strength = 0.06 * df_matched['precio_m2'].std()  # €/m2 (y)

        np.random.seed(42)
        jitter_x = np.random.uniform(-jitter_strength, jitter_strength, df_matched.shape[0])
        jitter_y = np.random.uniform(-jitter_y_strength, jitter_y_strength, df_matched.shape[0])

        fig.data[0].x = df_matched['superficie_m2'] + jitter_x
        fig.data[0].y = df_matched['precio_m2'] + jitter_y

        if len(fig.data) > 1:
            fig.data[1].x = df_matched['superficie_m2'][df_matched['es_outlier']] + jitter_x[df_matched['es_outlier']]
            fig.data[1].y = df_matched['precio_m2'][df_matched['es_outlier']] + jitter_y[df_matched['es_outlier']]

        # --- FIN jitter manual ---

        # Determinar rango custom de y para dejar margen visual arriba (para leyenda/anotacion)
        y_min = max(0, df_matched['precio_m2'].min() - 0.02 * df_matched['precio_m2'].std())
        y_max = df_matched['precio_m2'].max() + 0.13 * df_matched['precio_m2'].std()  # ampliar margen superior

        fig.add_vrect(
            x0=lb_sup, x1=ub_sup, fillcolor="rgba(0,200,80,0.08)", line_width=0,
            annotation_text="Zona normal", annotation_position="top left"
        )
        fig.add_vline(
            x=lb_sup, line_dash="dash", line_color="orange",
            annotation_text=f"Límite inferior ({lb_sup:.0f} m²)", 
            annotation_position="top left",
            annotation_font_color="orange"
        )
        fig.add_vline(
            x=ub_sup, line_dash="dash", line_color="orange",
            annotation_text=f"Límite superior ({ub_sup:.0f} m²)",
            annotation_position="top right",
            annotation_font_color="orange"
        )
        fig.update_layout(
            legend_title_text='Es outlier',
            legend=dict(
                yanchor="top", y=0.96,
                xanchor="left", x=0.01,
                title_font=dict(size=14),
                font=dict(size=13),
                bgcolor="rgba(255,255,255,0.95)",
                bordercolor="gray",
                borderwidth=1,
            ),
            xaxis=dict(
                tickformat=",d",
                gridcolor='rgba(200,200,200,0.15)',
                title_font=dict(size=16, color='gray'),
            ),
            yaxis=dict(
                gridcolor='rgba(200,200,200,0.10)',
                title_font=dict(size=16, color='gray'),
                range=[y_min, y_max]
            ),
            margin=dict(l=60, r=30, t=90, b=80),  # espacio mayor abajo al eje y
            plot_bgcolor="#fff",
            height=600  # aumentar altura del plot
        )

        # Ajustar separación de anotaciones respecto al eje x/y para mejor visibilidad
        # Ajustar el texto al ancho del contenedor (width) usando saltos de línea HTML <br>
        # y limitando el ancho visual (530 px). No se debe usar 'wrap', ya que Plotly no lo soporta.
        # Si el texto es demasiado largo, forzar saltos lógicos para mejorar legibilidad.
        texto_anotacion = (
            "🔎 <b>Outliers:</b> puntos rojos fuera del rango normal."
            f"<b>{len(outliers_sup)} de {len(df_matched)}</b> inmuebles <br> están en esta categoría."
            "<span style='color:#555'>"
            "Revisa el tooltip para detalles (barrio, año, m²...)</span>"
        )
        fig.add_annotation(
            text=texto_anotacion,
            showarrow=False, align="left",
            xref="paper", yref="paper", x=0.02, y=-0.15,
            width=530, 
            bgcolor="white", bordercolor="gray", borderwidth=1,
            font=dict(size=13, color="black")
            # 'wrap' no está soportado en Plotly, se ignora
        )
        fig.show()
        
        # Mostrar tabla ordenada y legible
        print("\n📋 Detalles de outliers en superficie (ordenados por tamaño descendente):")
        outlier_cols = [
            'superficie_m2', 'precio_m2', 'precio', 'habitaciones', 'banos',
            'barrio_id', 'ano_construccion'
        ]
        outlier_cols = [c for c in outlier_cols if c in outliers_sup.columns]
        if outlier_cols:
            print(outliers_sup[outlier_cols].sort_values("superficie_m2", ascending=False).to_string(index=False))
        else:
            print("No se encontraron columnas relevantes para mostrar detalles.")
        
        # Análisis de impacto resumido y gráfico gauge
        print("\n📊 Impacto de outliers en el dataset:")
        outlier_pct = len(outliers_sup) / len(df_matched) * 100
        sup_outliers = outliers_sup['superficie_m2'].sum()
        sup_total = df_matched['superficie_m2'].sum()
        pct_sup = sup_outliers / sup_total * 100 if sup_total > 0 else 0
        print(f"  Representan {outlier_pct:.1f}% del dataset")
        print(f"  Superficie total outliers: {sup_outliers:.0f} m² / {sup_total:.0f} m² "
              f"({pct_sup:.1f}%)")

        # Visual gauge de % outliers para reforzar narrativa
        gauge_fig = go.Figure(go.Indicator(
            mode="gauge+number",
            value=outlier_pct,
            number={'suffix': "%", 'font': {'size': 26}},
            title={
                'text': (
                    "Porcentaje de outliers<br>"
                    "<span style='font-size:14px; color:#2294a8'>vs. total registros</span>"
                ),
                'font': {'size': 14, 'color': '#7A3E65'}  # Fuente más pequeña y color distinto para el título principal
            },
            gauge={
                'axis': {'range': [None, 20], 'tickwidth': 1, 'tickcolor': "gray"},
                'bar': {'color': "#d62728"},
                'bgcolor': "white",
                'steps': [
                    {'range': [0, 5], 'color': '#e7f6e7'},
                    {'range': [5, 10], 'color': '#fff4b3'},
                    {'range': [10, 20], 'color': '#fad8d9'}
                ],
                'threshold': {
                    'line': {'color': "orange", 'width': 3},
                    'thickness': 0.75,
                    'value': outlier_pct
                }
            }))
        gauge_fig.update_layout(margin=dict(l=40, r=40, t=70, b=30))
        gauge_fig.show()

        # Recomendación explícita basada en perfil de outliers
        print("\n💡 Recomendación para tratamiento de outliers:")
        if outliers_sup['superficie_m2'].min() > 200:
            print("  - Todos los outliers son >200 m² (muy grandes para pisos típicos en BCN).")
            print("  - Analizar si son errores de alta en Catastro (ver casos duplicados).")
            print("  - Si son reales (áticos, dúplex, locales rehabilitados), filtrar solo si afectan el modelo o considerar transformación logarítmica para mitigarlos.")
        else:
            print("  - Hay mezcla de outliers pequeños y grandes.")
            print("  - Recomendada revisión manual y análisis caso por caso previo a filtrar o imputar.")
            print("  - Considera crear una variable dummy de outlier para control en modelos de regresión.")


⚠️  Outliers en precio_m2: 0 observaciones
   Rango normal esperado: [2571.98, 6859.40] €/m²

⚠️  Outliers en superficie_m2: 4 observaciones
   Rango normal esperado: [-80.38, 204.62] m²



📋 Detalles de outliers en superficie (ordenados por tamaño descendente):
 superficie_m2   precio_m2  precio  habitaciones  banos  barrio_id  ano_construccion
         473.0 4456.644820 2107993             4      3         32            1966.0
         380.0 4342.371053 1650101             2      1         28            1970.0
         324.0 4271.123457 1383844             2      1         32            2017.0
         289.0 4688.404844 1354949             2      1         32            1973.0

📊 Impacto de outliers en el dataset:
  Representan 4.0% del dataset
  Superficie total outliers: 1466 m² / 7781 m² (18.8%)



💡 Recomendación para tratamiento de outliers:
  - Todos los outliers son >200 m² (muy grandes para pisos típicos en BCN).
  - Analizar si son errores de alta en Catastro (ver casos duplicados).
  - Si son reales (áticos, dúplex, locales rehabilitados), filtrar solo si afectan el modelo o considerar transformación logarítmica para mitigarlos.


## 8. Insights y Conclusiones


In [66]:
import pandas as pd
import numpy as np
from IPython.display import display, Markdown

def generar_insights_pro(df):
    insights = []
    
    # 1. Análisis de Valor (Referencia Catastral y Mercado)
    if 'precio_m2' in df.columns:
        p_min, p_max, p_mean = df['precio_m2'].agg(['min', 'max', 'mean'])
        cv = (df['precio_m2'].std() / p_mean) * 100 # Coeficiente de variación
        insights.append(f"💰 **Dinámica de Precios:** Rango de **{p_min:,.0f}** a **{p_max:,.0f} €/m²**. La media se sitúa en **{p_mean:,.0f} €/m²** con una variabilidad (CV) del **{cv:.1f}%**.")

    # 2. Disparidad Geográfica (Barrios)
    if all(c in df.columns for c in ['barrio_nombre', 'precio_m2']):
        precios_barrio = df.groupby('barrio_nombre')['precio_m2'].mean().sort_values()
        brecha = (precios_barrio.iloc[-1] / precios_barrio.iloc[0] - 1) * 100
        insights.append(f"🏘️ **Brecha Territorial:** El barrio más costoso es **{precios_barrio.index[-1]}**, superando en un **{brecha:.1f}%** al más económico (**{precios_barrio.index[0]}**).")

    # 3. Factor Antigüedad (ACO - Catastro)
    if 'ano_construccion' in df.columns:
        # Según versión 1.5 del Catastro, la etiqueta <ACO> es clave para la valoración
        corr_edad = df[['precio_m2', 'ano_construccion']].corr().iloc[0, 1]
        edad_media = 2025 - df['ano_construccion'].mean()
        insights.append(f"🏗️ **Antigüedad (<ACO>):** La edad media del parque inmobiliario es de **{edad_media:.1f} años**. La correlación con el precio es de **{corr_edad:.3f}**.")

    # 4. Calidad de la Muestra (Outliers e Integridad)
    if 'superficie_m2' in df.columns:
        # Detectar outliers según superficie construida (sup) del Catastro
        q1, q3 = df['superficie_m2'].quantile([0.25, 0.75])
        iqr = q3 - q1
        outliers = df[(df['superficie_m2'] < q1 - 1.5*iqr) | (df['superficie_m2'] > q3 + 1.5*iqr)]
        pct_out = (len(outliers) / len(df)) * 100
        insights.append(f"⚠️ **Integridad de Datos:** Se han detectado **{len(outliers)}** propiedades fuera de rango (atipicidad: **{pct_out:.1f}%**).")

    # 5. Diagnóstico de Atributos Catastrales
    # Se valida la presencia de etiquetas clave: superficie (sup), uso (uso) y año (aco)
    etiquetas_catastro = ['precio_m2', 'superficie_m2', 'ano_construccion', 'habitaciones']
    completitud = df[etiquetas_catastro].notna().mean() * 100
    resumen_c = ", ".join([f"{k}: {v:.0f}%" for k, v in completitud.items()])
    insights.append(f"✅ **Completitud Catastral:** {resumen_c}.")

    # Formateo de Salida en Markdown para legibilidad total (con padding)
    output = "### 💡 INSIGHTS ESTRATÉGICOS DEL MERCADO (EDA)\n"
    output += "---\n"
    for i, line in enumerate(insights, 1):
        output += f"{i}. {line}\n\n"
    
    display(Markdown(output))

# Ejecución
generar_insights_pro(df_matched)

### 💡 INSIGHTS ESTRATÉGICOS DEL MERCADO (EDA)
---
1. 💰 **Dinámica de Precios:** Rango de **3,302** a **6,361 €/m²**. La media se sitúa en **4,679 €/m²** con una variabilidad (CV) del **14.1%**.

2. 🏘️ **Brecha Territorial:** El barrio más costoso es **la Vila de Gràcia**, superando en un **26.7%** al más económico (**el Coll**).

3. 🏗️ **Antigüedad (<ACO>):** La edad media del parque inmobiliario es de **50.0 años**. La correlación con el precio es de **0.212**.

4. ⚠️ **Integridad de Datos:** Se han detectado **4** propiedades fuera de rango (atipicidad: **4.0%**).

5. ✅ **Completitud Catastral:** precio_m2: 100%, superficie_m2: 100%, ano_construccion: 100%, habitaciones: 100%.



## 9. Recomendaciones para el Modelo


In [67]:
# Generar recomendaciones basadas en el EDA
recomendaciones = []

# Recomendación 1: Limpieza de outliers
if 'superficie_m2' in df_matched.columns:
    outliers_sup, lb_sup, ub_sup = detect_outliers_iqr(df_matched, 'superficie_m2')
    if len(outliers_sup) > 5:
        recomendaciones.append(f"🧹 Limpiar outliers en superficie: {len(outliers_sup)} observaciones fuera de rango [{lb_sup:.0f}, {ub_sup:.0f}] m²")

# Recomendación 2: Transformaciones
if 'superficie_m2' in df_matched.columns:
    corr_sup = df_matched[['precio_m2', 'superficie_m2']].corr().iloc[0, 1]
    if abs(corr_sup) < 0.2:
        recomendaciones.append("📈 Considerar transformaciones logarítmicas para mejorar relaciones lineales")

# Recomendación 3: Features importantes
if all(c in df_matched.columns for c in ['precio_m2', 'superficie_m2', 'ano_construccion', 'habitaciones']):
    corrs = {
        'superficie_m2': df_matched[['precio_m2', 'superficie_m2']].corr().iloc[0, 1],
        'ano_construccion': df_matched[['precio_m2', 'ano_construccion']].corr().iloc[0, 1],
        'habitaciones': df_matched[['precio_m2', 'habitaciones']].corr().iloc[0, 1]
    }
    best_feature = max(corrs.items(), key=lambda x: abs(x[1]))
    recomendaciones.append(f"⭐ Feature más correlacionada: {best_feature[0]} (corr={best_feature[1]:.3f})")

# Recomendación 4: Tamaño de muestra
if len(df_matched) < 150:
    recomendaciones.append(f"📊 Tamaño de muestra pequeño ({len(df_matched)} obs). Considerar aumentar datos o usar cross-validation")

print("\n" + "="*70)
print("💡 RECOMENDACIONES PARA EL MODELO")
print("="*70)
for i, rec in enumerate(recomendaciones, 1):
    print(f"{i}. {rec}")
if not recomendaciones:
    print("✅ No se detectaron problemas críticos")
print("="*70)



💡 RECOMENDACIONES PARA EL MODELO
1. 📈 Considerar transformaciones logarítmicas para mejorar relaciones lineales
2. ⭐ Feature más correlacionada: habitaciones (corr=-0.223)
3. 📊 Tamaño de muestra pequeño (100 obs). Considerar aumentar datos o usar cross-validation


## 10. Análisis de Transformaciones (Recomendación 1)

Basado en las recomendaciones, exploramos transformaciones logarítmicas para mejorar las relaciones lineales.


In [69]:
# Aplicar transformaciones logarítmicas
df_log = df_matched.copy()

# Crear variables transformadas
if 'superficie_m2' in df_log.columns:
    df_log['log_superficie'] = np.log(df_log['superficie_m2'] + 1)  # +1 para evitar log(0)

if 'precio_m2' in df_log.columns:
    df_log['log_precio_m2'] = np.log(df_log['precio_m2'])

# Comparar correlaciones antes y después
print("📊 COMPARACIÓN DE CORRELACIONES")
print("="*70)

if 'superficie_m2' in df_log.columns and 'precio_m2' in df_log.columns:
    corr_original = df_log[['precio_m2', 'superficie_m2']].corr().iloc[0, 1]
    corr_log = df_log[['log_precio_m2', 'log_superficie']].corr().iloc[0, 1]
    
    print(f"\nSuperficie vs Precio/m²:")
    print(f"  Original: {corr_original:.3f}")
    print(f"  Log-transformado: {corr_log:.3f}")
    print(f"  Mejora: {abs(corr_log) - abs(corr_original):+.3f}")

# Visualizar relación log-transformada
if 'log_superficie' in df_log.columns and 'log_precio_m2' in df_log.columns:
    fig = px.scatter(
        df_log,
        x='log_superficie',
        y='log_precio_m2',
        color='barrio_id' if 'barrio_id' in df_log.columns else None,
        title='Relación Log-transformada: log(Precio/m²) vs log(Superficie)',
        labels={'log_superficie': 'log(Superficie + 1)', 'log_precio_m2': 'log(Precio/m²)'}
    )
    fig.show()
    
    print("\n💡 Interpretación:")
    if abs(corr_log) > abs(corr_original):
        print("  ✅ La transformación log mejora la relación lineal")
        print("  → Considerar usar log(superficie) y log(precio_m2) en el modelo")
    else:
        print("  ⚠️  La transformación log no mejora significativamente")
        print("  → Mantener variables originales o probar otras transformaciones")


📊 COMPARACIÓN DE CORRELACIONES

Superficie vs Precio/m²:
  Original: -0.091
  Log-transformado: -0.089
  Mejora: -0.002



💡 Interpretación:
  ⚠️  La transformación log no mejora significativamente
  → Mantener variables originales o probar otras transformaciones


## 11. Análisis Detallado: Habitaciones (Recomendación 2)

La feature más correlacionada es `habitaciones` (corr=-0.223). Analizamos esta relación en detalle.


In [70]:
# Análisis detallado de la relación habitaciones vs precio
if 'habitaciones' in df_matched.columns and 'precio_m2' in df_matched.columns:
    print("🔍 ANÁLISIS: HABITACIONES vs PRECIO/M²")
    print("="*70)
    
    # Estadísticas por número de habitaciones
    stats_hab = df_matched.groupby('habitaciones').agg({
        'precio_m2': ['mean', 'std', 'count', 'min', 'max'],
        'superficie_m2': ['mean', 'std'] if 'superficie_m2' in df_matched.columns else None,
        'precio': ['mean'] if 'precio' in df_matched.columns else None
    }).round(2)
    
    print("\n📊 Estadísticas por habitaciones:")
    print(stats_hab)
    
    # Correlación
    corr_hab = df_matched[['precio_m2', 'habitaciones']].corr().iloc[0, 1]
    print(f"\n📈 Correlación precio_m2 vs habitaciones: {corr_hab:.3f}")
    
    print("\n💡 Interpretación:")
    if corr_hab < 0:
        print("  ⚠️  Correlación NEGATIVA: Más habitaciones → Menor precio/m²")
        print("  Posibles explicaciones:")
        print("    • Pisos más grandes (más habitaciones) tienen menor precio/m²")
        print("    • Efecto de 'descuento por volumen' en el mercado")
        print("    • Datos mock pueden tener relaciones artificiales")
    else:
        print("  ✅ Correlación POSITIVA: Más habitaciones → Mayor precio/m²")
    
    # Visualización mejorada
    fig = px.scatter(
        df_matched,
        x='habitaciones',
        y='precio_m2',
        size='superficie_m2' if 'superficie_m2' in df_matched.columns else None,
        color='barrio_id' if 'barrio_id' in df_matched.columns else None,
        hover_data=['superficie_m2', 'precio', 'banos'] if all(c in df_matched.columns for c in ['superficie_m2', 'precio', 'banos']) else None,
        title='Precio/m² vs Número de Habitaciones (con superficie)',
        labels={'habitaciones': 'Habitaciones', 'precio_m2': 'Precio/m² (€/m²)'}
    )
    fig.show()
    
    # Análisis de superficie por habitaciones
    if 'superficie_m2' in df_matched.columns:
        print("\n📐 Superficie media por habitaciones:")
        superficie_por_hab = df_matched.groupby('habitaciones')['superficie_m2'].mean()
        for hab, sup in superficie_por_hab.items():
            print(f"  {hab} hab: {sup:.1f} m²")


🔍 ANÁLISIS: HABITACIONES vs PRECIO/M²

📊 Estadísticas por habitaciones:
             precio_m2                                 superficie_m2          \
                  mean     std count      min      max          mean     std   
habitaciones                                                                   
1              5016.75  614.79    13  3447.97  5599.30         62.95   42.21   
2              4757.44  676.21    41  3478.93  6360.55         82.59   81.58   
3              4472.62  573.87    35  3302.42  5618.51         66.80   39.73   
4              4640.14  743.41    11  3411.31  5715.81        112.64  126.23   

                 precio  
                   mean  
habitaciones             
1             305634.85  
2             385169.02  
3             304334.09  
4             505444.64  

📈 Correlación precio_m2 vs habitaciones: -0.223

💡 Interpretación:
  ⚠️  Correlación NEGATIVA: Más habitaciones → Menor precio/m²
  Posibles explicaciones:
    • Pisos más grandes (más


📐 Superficie media por habitaciones:
  1 hab: 63.0 m²
  2 hab: 82.6 m²
  3 hab: 66.8 m²
  4 hab: 112.6 m²


## 12. Resumen Ejecutivo y Próximos Pasos


In [71]:
# Generar resumen ejecutivo completo
print("="*70)
print("📋 RESUMEN EJECUTIVO - EDA MODELO HEDONIC MICRO")
print("="*70)

print("\n1️⃣ DATOS:")
print(f"   • Observaciones: {len(df_matched)}")
print(f"   • Barrios: {df_matched['barrio_id'].nunique() if 'barrio_id' in df_matched.columns else 'N/A'}")
print(f"   • Completitud: {df_matched['precio_m2'].notna().sum()/len(df_matched)*100:.1f}%")

print("\n2️⃣ VARIABLES CLAVE:")
if 'precio_m2' in df_matched.columns:
    print(f"   • Precio/m²: {df_matched['precio_m2'].mean():.0f} €/m² (rango: {df_matched['precio_m2'].min():.0f}-{df_matched['precio_m2'].max():.0f})")
if 'superficie_m2' in df_matched.columns:
    print(f"   • Superficie: {df_matched['superficie_m2'].mean():.1f} m² (rango: {df_matched['superficie_m2'].min():.1f}-{df_matched['superficie_m2'].max():.1f})")

print("\n3️⃣ CORRELACIONES:")
numeric_cols = ['superficie_m2', 'ano_construccion', 'habitaciones', 'banos']
numeric_cols = [c for c in numeric_cols if c in df_matched.columns]
if numeric_cols:
    corrs = df_matched[['precio_m2'] + numeric_cols].corr()['precio_m2'].sort_values(ascending=False)
    for var, corr in corrs.items():
        if var != 'precio_m2':
            print(f"   • {var:20s}: {corr:7.3f}")

print("\n4️⃣ OUTLIERS:")
if 'superficie_m2' in df_matched.columns:
    Q1 = df_matched['superficie_m2'].quantile(0.25)
    Q3 = df_matched['superficie_m2'].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df_matched[(df_matched['superficie_m2'] < Q1 - 1.5*IQR) | (df_matched['superficie_m2'] > Q3 + 1.5*IQR)]
    print(f"   • Superficie: {len(outliers)} observaciones ({len(outliers)/len(df_matched)*100:.1f}%)")

print("\n5️⃣ RECOMENDACIONES PARA EL MODELO:")
print("   ✅ Usar transformación logarítmica para superficie y precio")
print("   ✅ Considerar habitaciones como feature principal (aunque correlación negativa)")
print("   ✅ Filtrar outliers en superficie (>200 m²) o usar transformación log")
print("   ✅ Usar cross-validation (5-fold) en vez de train/test split")
print("   ⚠️  Esperar datos reales de Idealista para validar relaciones")

print("\n6️⃣ PRÓXIMOS PASOS:")
print("   1. Limpiar outliers o aplicar transformaciones")
print("   2. Entrenar modelo con variables transformadas")
print("   3. Comparar modelo log vs original")
print("   4. Validar con datos reales cuando estén disponibles")

print("\n" + "="*70)
print("📝 NOTA: Estos resultados son con datos mock. Relaciones reales pueden diferir.")
print("="*70)


📋 RESUMEN EJECUTIVO - EDA MODELO HEDONIC MICRO

1️⃣ DATOS:
   • Observaciones: 100
   • Barrios: 5
   • Completitud: 100.0%

2️⃣ VARIABLES CLAVE:
   • Precio/m²: 4679 €/m² (rango: 3302-6361)
   • Superficie: 77.8 m² (rango: 2.9-473.0)

3️⃣ CORRELACIONES:
   • ano_construccion    :   0.212
   • superficie_m2       :  -0.091
   • banos               :  -0.181
   • habitaciones        :  -0.223

4️⃣ OUTLIERS:
   • Superficie: 4 observaciones (4.0%)

5️⃣ RECOMENDACIONES PARA EL MODELO:
   ✅ Usar transformación logarítmica para superficie y precio
   ✅ Considerar habitaciones como feature principal (aunque correlación negativa)
   ✅ Filtrar outliers en superficie (>200 m²) o usar transformación log
   ✅ Usar cross-validation (5-fold) en vez de train/test split
   ⚠️  Esperar datos reales de Idealista para validar relaciones

6️⃣ PRÓXIMOS PASOS:
   1. Limpiar outliers o aplicar transformaciones
   2. Entrenar modelo con variables transformadas
   3. Comparar modelo log vs original
   4. Vali

---

## 📝 Notas Finales

- **Datos mock**: Los datos de Idealista son simulados. Resultados pueden no reflejar relaciones reales del mercado.
- **Tamaño muestra**: 100 observaciones es el mínimo para modelos hedonic. Considerar aumentar cuando haya datos reales.
- **Outliers**: Algunos valores extremos pueden requerir limpieza antes de entrenar el modelo.
- **Correlaciones bajas**: Las correlaciones observadas pueden deberse a datos mock. Validar con datos reales.

---

**Última actualización**: 2025-12-19


In [72]:
# Análisis de interacciones: Superficie × Barrio
if all(c in df_matched.columns for c in ['superficie_m2', 'precio_m2', 'barrio_id']):
    print("🔍 ANÁLISIS DE INTERACCIONES: SUPERFICIE × BARRIO")
    print("="*70)
    
    # Crear grupos de superficie (pequeño, mediano, grande)
    df_matched['superficie_categoria'] = pd.cut(
        df_matched['superficie_m2'],
        bins=[0, 60, 90, 200, float('inf')],
        labels=['<60 m²', '60-90 m²', '90-200 m²', '>200 m²']
    )
    
    # Precio/m² medio por combinación superficie × barrio
    interaction = df_matched.groupby(['barrio_id', 'superficie_categoria'])['precio_m2'].agg(['mean', 'count']).reset_index()
    interaction_pivot = interaction.pivot(index='barrio_id', columns='superficie_categoria', values='mean')
    
    print("\n📊 Precio/m² medio por Barrio × Categoría de Superficie:")
    print(interaction_pivot.round(0))
    
    # Visualización
    fig = px.bar(
        interaction,
        x='barrio_id',
        y='mean',
        color='superficie_categoria',
        barmode='group',
        title='Interacción: Precio/m² por Barrio × Categoría de Superficie',
        labels={'mean': 'Precio/m² Medio (€/m²)', 'barrio_id': 'Barrio', 'superficie_categoria': 'Categoría Superficie'},
        hover_data=['count']
    )
    fig.show()
    
    print("\n💡 Interpretación:")
    print("  - Si hay diferencias grandes entre categorías dentro del mismo barrio,")
    print("    → La interacción superficie×barrio puede ser útil en el modelo")
    print("  - Si las diferencias son pequeñas,")
    print("    → Las variables independientes pueden ser suficientes")


🔍 ANÁLISIS DE INTERACCIONES: SUPERFICIE × BARRIO

📊 Precio/m² medio por Barrio × Categoría de Superficie:
superficie_categoria  <60 m²  60-90 m²  90-200 m²  >200 m²
barrio_id                                                 
28                    4404.0       NaN        NaN   4342.0
29                    3940.0    3911.0     4520.0      NaN
30                    4718.0    4706.0     4308.0      NaN
31                    5202.0    4750.0     5284.0      NaN
32                    4620.0    4694.0     5021.0   4472.0


/var/folders/8t/1rzjlj6n3rj3v08skl2pwjbm0000gn/T/ipykernel_69290/2787252725.py:14: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.




💡 Interpretación:
  - Si hay diferencias grandes entre categorías dentro del mismo barrio,
    → La interacción superficie×barrio puede ser útil en el modelo
  - Si las diferencias son pequeñas,
    → Las variables independientes pueden ser suficientes


In [73]:
# Análisis de características combinadas (ascensor, exterior)
if all(c in df_matched.columns for c in ['ascensor', 'exterior', 'precio_m2']):
    print("🔍 ANÁLISIS DE CARACTERÍSTICAS COMBINADAS")
    print("="*70)
    
    # Convertir a boolean si no lo son
    df_comb = df_matched.copy()
    if df_comb['ascensor'].dtype == 'object':
        df_comb['ascensor'] = df_comb['ascensor'].astype(bool)
    if df_comb['exterior'].dtype == 'object':
        df_comb['exterior'] = df_comb['exterior'].astype(bool)
    
    # Crear combinaciones
    df_comb['caracteristicas'] = df_comb.apply(
        lambda row: f"Ascensor: {'Sí' if row['ascensor'] else 'No'}, Exterior: {'Sí' if row['exterior'] else 'No'}",
        axis=1
    )
    
    stats_comb = df_comb.groupby('caracteristicas')['precio_m2'].agg(['mean', 'std', 'count']).round(2)
    print("\n📊 Precio/m² por combinación de características:")
    print(stats_comb)
    
    # Visualización
    fig = px.box(
        df_comb,
        x='caracteristicas',
        y='precio_m2',
        title='Distribución de Precio/m² por Combinación de Características',
        labels={'precio_m2': 'Precio/m² (€/m²)', 'caracteristicas': 'Características'}
    )
    fig.update_xaxes(tickangle=45)
    fig.show()
    
    # Análisis de efecto individual
    print("\n📈 Efecto individual de características:")
    if 'ascensor' in df_comb.columns:
        precio_con_ascensor = df_comb[df_comb['ascensor'] == True]['precio_m2'].mean()
        precio_sin_ascensor = df_comb[df_comb['ascensor'] == False]['precio_m2'].mean()
        print(f"  Ascensor: Con={precio_con_ascensor:.0f} €/m², Sin={precio_sin_ascensor:.0f} €/m², Diferencia={precio_con_ascensor-precio_sin_ascensor:.0f} €/m²")
    
    if 'exterior' in df_comb.columns:
        precio_exterior = df_comb[df_comb['exterior'] == True]['precio_m2'].mean()
        precio_interior = df_comb[df_comb['exterior'] == False]['precio_m2'].mean()
        print(f"  Exterior: Sí={precio_exterior:.0f} €/m², No={precio_interior:.0f} €/m², Diferencia={precio_exterior-precio_interior:.0f} €/m²")


🔍 ANÁLISIS DE CARACTERÍSTICAS COMBINADAS

📊 Precio/m² por combinación de características:
                               mean     std  count
caracteristicas                                   
Ascensor: No, Exterior: No  4599.44  673.92     45
Ascensor: No, Exterior: Sí  4706.15  688.95     37
Ascensor: Sí, Exterior: No  4792.80  467.72     11
Ascensor: Sí, Exterior: Sí  4861.90  688.63      7



📈 Efecto individual de características:
  Ascensor: Con=4820 €/m², Sin=4648 €/m², Diferencia=172 €/m²
  Exterior: Sí=4731 €/m², No=4637 €/m², Diferencia=94 €/m²


## 13. Análisis de Precio Total vs Precio/m² (Efecto de Escala)


In [74]:
# Análisis de relación precio total vs precio/m²
if all(c in df_matched.columns for c in ['precio', 'precio_m2', 'superficie_m2']):
    print("🔍 ANÁLISIS: PRECIO TOTAL vs PRECIO/M²")
    print("="*70)
    
    # Calcular ratio precio/precio_m2 (debería ser ≈ superficie)
    df_matched['precio_total_calculado'] = df_matched['precio_m2'] * df_matched['superficie_m2']
    df_matched['ratio_precio'] = df_matched['precio'] / df_matched['precio_total_calculado']
    
    print(f"\n📊 Validación de consistencia:")
    print(f"  Ratio medio precio/precio_m2×superficie: {df_matched['ratio_precio'].mean():.3f}")
    print(f"  (Debería ser ≈ 1.0 si los datos son consistentes)")
    
    # Visualización: Precio total vs Superficie
    fig = px.scatter(
        df_matched,
        x='superficie_m2',
        y='precio',
        color='precio_m2',
        size='habitaciones' if 'habitaciones' in df_matched.columns else None,
        hover_data=['barrio_id', 'precio_m2', 'habitaciones'] if all(c in df_matched.columns for c in ['barrio_id', 'precio_m2', 'habitaciones']) else None,
        title='Precio Total vs Superficie (coloreado por precio/m²)',
        labels={'superficie_m2': 'Superficie (m²)', 'precio': 'Precio Total (€)', 'precio_m2': 'Precio/m² (€/m²)'},
        color_continuous_scale='Viridis'
    )
    fig.show()
    
    # Análisis de elasticidad precio-superficie
    print("\n📈 Elasticidad Precio-Superficie:")
    print("  (¿Cómo cambia el precio total cuando aumenta la superficie?)")
    
    # Calcular correlación log-log (elasticidad)
    if 'superficie_m2' in df_log.columns and 'precio' in df_log.columns:
        df_log['log_precio_total'] = np.log(df_log['precio'])
        elasticidad = df_log[['log_precio_total', 'log_superficie']].corr().iloc[0, 1]
        print(f"  Correlación log(precio) vs log(superficie): {elasticidad:.3f}")
        print(f"  Interpretación: Si elasticidad ≈ 1, precio aumenta proporcionalmente con superficie")
        print(f"                   Si elasticidad < 1, hay 'descuento por volumen'")
        print(f"                   Si elasticidad > 1, hay 'premium por tamaño'")


🔍 ANÁLISIS: PRECIO TOTAL vs PRECIO/M²

📊 Validación de consistencia:
  Ratio medio precio/precio_m2×superficie: 1.000
  (Debería ser ≈ 1.0 si los datos son consistentes)



📈 Elasticidad Precio-Superficie:
  (¿Cómo cambia el precio total cuando aumenta la superficie?)
  Correlación log(precio) vs log(superficie): 0.989
  Interpretación: Si elasticidad ≈ 1, precio aumenta proporcionalmente con superficie
                   Si elasticidad < 1, hay 'descuento por volumen'
                   Si elasticidad > 1, hay 'premium por tamaño'


## 14. Heatmap de Correlaciones con Clustering


In [75]:
# Heatmap de correlaciones con clustering jerárquico
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.spatial.distance import squareform
import plotly.figure_factory as ff

numeric_cols = ['precio_m2', 'precio', 'superficie_m2', 'ano_construccion', 
                'plantas', 'habitaciones', 'banos']
numeric_cols = [c for c in numeric_cols if c in df_matched.columns]

if len(numeric_cols) > 2:
    corr_matrix = df_matched[numeric_cols].corr()
    
    # Crear heatmap con clustering
    fig = ff.create_dendrogram(
        corr_matrix.values,
        orientation='bottom',
        labels=corr_matrix.columns.tolist()
    )
    
    # Heatmap mejorado con valores
    fig = px.imshow(
        corr_matrix,
        text_auto='.2f',
        aspect="auto",
        title="Matriz de Correlaciones (con valores)",
        color_continuous_scale="RdBu",
        zmin=-1,
        zmax=1,
        labels=dict(color="Correlación")
    )
    fig.update_layout(height=600)
    fig.show()
    
    # Identificar grupos de variables correlacionadas
    print("\n🔗 GRUPOS DE VARIABLES CORRELACIONADAS:")
    print("="*70)
    
    # Variables altamente correlacionadas (|r| > 0.5)
    high_corr_pairs = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            corr_val = corr_matrix.iloc[i, j]
            if abs(corr_val) > 0.5:
                high_corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_val))
    
    if high_corr_pairs:
        print("\n⚠️  Variables altamente correlacionadas (|r| > 0.5):")
        for var1, var2, corr in sorted(high_corr_pairs, key=lambda x: abs(x[2]), reverse=True):
            print(f"  • {var1} ↔ {var2}: {corr:.3f}")
        print("\n💡 Recomendación: Considerar eliminar una de cada par para evitar multicolinealidad")
    else:
        print("\n✅ No hay pares de variables altamente correlacionadas (|r| > 0.5)")
        print("   → Menor riesgo de multicolinealidad en el modelo")



🔗 GRUPOS DE VARIABLES CORRELACIONADAS:

⚠️  Variables altamente correlacionadas (|r| > 0.5):
  • precio ↔ superficie_m2: 0.987
  • habitaciones ↔ banos: 0.928

💡 Recomendación: Considerar eliminar una de cada par para evitar multicolinealidad


## 15. Análisis Temporal (Año de Construcción)


In [76]:
# Análisis de precio por año de construcción
if 'ano_construccion' in df_matched.columns and 'precio_m2' in df_matched.columns:
    print("🔍 ANÁLISIS TEMPORAL: PRECIO vs AÑO DE CONSTRUCCIÓN")
    print("="*70)
    
    # Crear categorías de antigüedad
    df_matched['antiguedad'] = 2025 - df_matched['ano_construccion']
    df_matched['categoria_antiguedad'] = pd.cut(
        df_matched['antiguedad'],
        bins=[0, 10, 30, 50, 100, float('inf')],
        labels=['<10 años', '10-30 años', '30-50 años', '50-100 años', '>100 años']
    )
    
    # Precio/m² por categoría de antigüedad
    stats_antiguedad = df_matched.groupby('categoria_antiguedad')['precio_m2'].agg(['mean', 'std', 'count']).round(2)
    print("\n📊 Precio/m² por categoría de antigüedad:")
    print(stats_antiguedad)
    
    # Visualización: Precio/m² vs Año de construcción (con tendencia)
    fig = px.scatter(
        df_matched,
        x='ano_construccion',
        y='precio_m2',
        color='barrio_id' if 'barrio_id' in df_matched.columns else None,
        size='superficie_m2' if 'superficie_m2' in df_matched.columns else None,
        trendline="ols",  # Añadir línea de tendencia OLS
        title='Precio/m² vs Año de Construcción (con tendencia)',
        labels={'ano_construccion': 'Año de Construcción', 'precio_m2': 'Precio/m² (€/m²)'}
    )
    fig.show()
    
    # Análisis de efecto de renovación
    print("\n💡 Interpretación:")
    print("  - Si la tendencia es positiva: Edificios más nuevos son más caros")
    print("  - Si la tendencia es negativa: Edificios más antiguos son más caros (posible efecto 'charm')")
    print("  - Si la tendencia es plana: El año no afecta significativamente el precio")
    
    # Box plot por categoría de antigüedad
    fig = px.box(
        df_matched,
        x='categoria_antiguedad',
        y='precio_m2',
        title='Distribución de Precio/m² por Antigüedad del Edificio',
        labels={'precio_m2': 'Precio/m² (€/m²)', 'categoria_antiguedad': 'Antigüedad'}
    )
    fig.show()


🔍 ANÁLISIS TEMPORAL: PRECIO vs AÑO DE CONSTRUCCIÓN

📊 Precio/m² por categoría de antigüedad:
                         mean     std  count
categoria_antiguedad                        
<10 años              4359.99  635.74      7
10-30 años            5263.89  692.94     12
30-50 años            4508.87  630.48     25
50-100 años           4717.21  598.96     50
>100 años             4264.60  514.15      6


/var/folders/8t/1rzjlj6n3rj3v08skl2pwjbm0000gn/T/ipykernel_69290/2823204823.py:15: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.




💡 Interpretación:
  - Si la tendencia es positiva: Edificios más nuevos son más caros
  - Si la tendencia es negativa: Edificios más antiguos son más caros (posible efecto 'charm')
  - Si la tendencia es plana: El año no afecta significativamente el precio


## 16. Resumen Ejecutivo y Próximos Pasos


In [77]:
# Generar resumen ejecutivo completo
print("="*70)
print("📋 RESUMEN EJECUTIVO - EDA MODELO HEDONIC MICRO")
print("="*70)

print("\n1️⃣ DATOS:")
print(f"   • Observaciones: {len(df_matched)}")
print(f"   • Barrios: {df_matched['barrio_id'].nunique() if 'barrio_id' in df_matched.columns else 'N/A'}")
print(f"   • Completitud: {df_matched['precio_m2'].notna().sum()/len(df_matched)*100:.1f}%")

print("\n2️⃣ VARIABLES CLAVE:")
if 'precio_m2' in df_matched.columns:
    print(f"   • Precio/m²: {df_matched['precio_m2'].mean():.0f} €/m² (rango: {df_matched['precio_m2'].min():.0f}-{df_matched['precio_m2'].max():.0f})")
if 'superficie_m2' in df_matched.columns:
    print(f"   • Superficie: {df_matched['superficie_m2'].mean():.1f} m² (rango: {df_matched['superficie_m2'].min():.1f}-{df_matched['superficie_m2'].max():.1f})")

print("\n3️⃣ CORRELACIONES:")
numeric_cols = ['superficie_m2', 'ano_construccion', 'habitaciones', 'banos']
numeric_cols = [c for c in numeric_cols if c in df_matched.columns]
if numeric_cols:
    corrs = df_matched[['precio_m2'] + numeric_cols].corr()['precio_m2'].sort_values(ascending=False)
    for var, corr in corrs.items():
        if var != 'precio_m2':
            print(f"   • {var:20s}: {corr:7.3f}")

print("\n4️⃣ OUTLIERS:")
if 'superficie_m2' in df_matched.columns:
    Q1 = df_matched['superficie_m2'].quantile(0.25)
    Q3 = df_matched['superficie_m2'].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df_matched[(df_matched['superficie_m2'] < Q1 - 1.5*IQR) | (df_matched['superficie_m2'] > Q3 + 1.5*IQR)]
    print(f"   • Superficie: {len(outliers)} observaciones ({len(outliers)/len(df_matched)*100:.1f}%)")

print("\n5️⃣ HALLAZGOS CLAVE:")
print("   ✅ Interacciones superficie×barrio pueden ser útiles")
print("   ✅ Características combinadas (ascensor, exterior) muestran diferencias")
print("   ✅ Año de construcción tiene efecto en precio")
print("   ⚠️  Correlaciones bajas pueden deberse a datos mock")

print("\n6️⃣ RECOMENDACIONES PARA EL MODELO:")
print("   ✅ Usar transformación logarítmica para superficie y precio")
print("   ✅ Incluir interacciones: superficie×barrio, año×barrio")
print("   ✅ Considerar características combinadas (ascensor, exterior)")
print("   ✅ Filtrar outliers en superficie (>200 m²) o usar transformación log")
print("   ✅ Usar cross-validation (5-fold) en vez de train/test split")
print("   ⚠️  Esperar datos reales de Idealista para validar relaciones")

print("\n7️⃣ PRÓXIMOS PASOS:")
print("   1. Limpiar outliers o aplicar transformaciones")
print("   2. Entrenar modelo con variables transformadas e interacciones")
print("   3. Comparar modelo log vs original")
print("   4. Validar con datos reales cuando estén disponibles")

print("\n" + "="*70)
print("📝 NOTA: Estos resultados son con datos mock. Relaciones reales pueden diferir.")
print("="*70)


📋 RESUMEN EJECUTIVO - EDA MODELO HEDONIC MICRO

1️⃣ DATOS:
   • Observaciones: 100
   • Barrios: 5
   • Completitud: 100.0%

2️⃣ VARIABLES CLAVE:
   • Precio/m²: 4679 €/m² (rango: 3302-6361)
   • Superficie: 77.8 m² (rango: 2.9-473.0)

3️⃣ CORRELACIONES:
   • ano_construccion    :   0.212
   • superficie_m2       :  -0.091
   • banos               :  -0.181
   • habitaciones        :  -0.223

4️⃣ OUTLIERS:
   • Superficie: 4 observaciones (4.0%)

5️⃣ HALLAZGOS CLAVE:
   ✅ Interacciones superficie×barrio pueden ser útiles
   ✅ Características combinadas (ascensor, exterior) muestran diferencias
   ✅ Año de construcción tiene efecto en precio
   ⚠️  Correlaciones bajas pueden deberse a datos mock

6️⃣ RECOMENDACIONES PARA EL MODELO:
   ✅ Usar transformación logarítmica para superficie y precio
   ✅ Incluir interacciones: superficie×barrio, año×barrio
   ✅ Considerar características combinadas (ascensor, exterior)
   ✅ Filtrar outliers en superficie (>200 m²) o usar transformación log
   